In [1]:
# Refactoring NASEM model
import nasem_dairy as nd
import pandas as pd
import math
from pprint import pprint

def run_NASEM():
########################################
# Step 1: Read User Input
########################################
    # animal_input is a dictionary with all animal specific parameters
    # diet_info is a dataframe with the user entered feed ingredients and %DM intakes
    user_diet, animal_input, equation_selection = nd.read_csv_input('./input.csv')
        
    # list_of_feeds is used to query the database and retrieve the ingredient composition, stored in feed_data
    list_of_feeds = user_diet['Feedstuff'].tolist()
    feed_data = nd.fl_get_rows(list_of_feeds, '../../src/nasem_dairy/data/diet_database.db')
    feed_data.reset_index(inplace=True)
    feed_data = feed_data.rename(columns={"Fd_Name": "Feedstuff"})

    diet_info_initial = pd.DataFrame({'Feedstuff': user_diet['Feedstuff']})
    diet_info_initial = diet_info_initial.merge(feed_data, how='left', on='Feedstuff')

    # Add Fd_DMInp to the diet_info dataframe
    Fd_DMInp = user_diet.set_index('Feedstuff')['kg_user'] / user_diet['kg_user'].sum()
    diet_info_initial.insert(1, 'Fd_DMInp', Fd_DMInp.reindex(diet_info_initial['Feedstuff']).values)
    diet_info_initial['Fd_DMIn'] = diet_info_initial['Fd_DMInp'] * animal_input['DMI']      # Should this be done after DMI equations?

    # Add Fd_DNDF48 column, need to add to the database
    diet_info_initial['Fd_DNDF48'] = 0  

    # Calculate additional physiology values
    animal_input['An_PrePartDay'] = animal_input['An_GestDay'] - animal_input['An_GestLength']
    animal_input['An_PrePartWk'] = animal_input['An_PrePartDay'] / 7

    del(list_of_feeds, Fd_DMInp)

    # Import infusion data
    if equation_selection['use_infusions'] == 0:
        pass
    elif equation_selection['use_infusions'] == 1:
        infusions = nd.read_infusion_input('./infusions.csv')
    else:
        raise ValueError(f"Invalid use_infusions: {equation_selection['use_infusions']} was entered. Must be 0 or 1")


########################################
# Step 2: DMI Equations
########################################
    # # Need to precalculate Dt_NDF for DMI predicitons, this will be based on the user entered DMI (animal_input['DMI])
    Dt_NDF = (diet_info_initial['Fd_NDF'] * diet_info_initial['Fd_DMInp']).sum()

    if equation_selection['DMIn_eqn'] == 0:
        # print('Using user input DMI')
        pass

    # Predict DMI for lactating cow
    elif equation_selection['DMIn_eqn'] == 8: 
        # print("using DMIn_eqn: 8")
        animal_input['DMI'] = nd.calculate_Dt_DMIn_Lact1(
            animal_input['An_Parity_rl'], 
            animal_input['Trg_MilkProd'], 
            animal_input['An_BW'], 
            animal_input['An_BCS'],
            animal_input['An_LactDay'], 
            animal_input['Trg_MilkFatp'], 
            animal_input['Trg_MilkTPp'], 
            animal_input['Trg_MilkLacp'])

    # Predict DMI for heifers    
    elif equation_selection['DMIn_eqn'] in [2,3,4,5,6,7,12,13,14,15,16,17]:
        animal_input['DMI'] = nd.heifer_growth(
            equation_selection['DMIn_eqn'], 
            # diet_info.loc['Diet', 'Fd_NDF'],
            Dt_NDF, 
            animal_input['An_BW'], 
            animal_input['An_BW_mature'], 
            animal_input['An_PrePartWk'], 
            nd.coeff_dict)

    
    elif equation_selection['DMIn_eqn'] in [10,11]:
        animal_input['DMI'] = nd.dry_cow_equations(
            equation_selection['DMIn_eqn'], 
            animal_input['An_BW'], 
            animal_input['An_PrePartWk'], 
            animal_input['An_GestDay'], 
            animal_input['An_GestLength'], 
            Dt_NDF, 
            nd.coeff_dict)
        
    else:
        # It needs to catch all possible solutions, otherwise it's possible that it stays unchanged without warning
        print("DMIn_eqn uncaught - DMI not changed. equation_selection[DMIn_eqn]: "+ str(equation_selection['DMIn_eqn']))

    del(Dt_NDF)

########################################
# Step 3: Feed Based Calculations
########################################
    diet_info = nd.calculate_diet_info(animal_input['DMI'],
                                       animal_input['An_StatePhys'],
                                       equation_selection['Use_DNDF_IV'],
                                       diet_info=diet_info_initial,
                                       coeff_dict=nd.coeff_dict)
    # All equations in the f dataframe go into calculate_diet_info()
    # This includes micronutrient calculations which are no longer handled by seperate functions
    
    diet_data = nd.calculate_diet_data(diet_info, 
                                       animal_input['DMI'], 
                                       animal_input['An_BW'],
                                       nd.coeff_dict)
    # diet_data contains everything starting with "Dt_"

########################################
# Step 4: Infusion Calculations
########################################
    infusion_data = nd.calculate_infusion_data(infusions,
                                               animal_input['DMI'],
                                               nd.coeff_dict)
    
########################################
# Step 5: Rumen Digestion Calculations
########################################
    # Rumen Digestability Coefficients
    Rum_dcNDF = nd.calculate_Rum_dcNDF(animal_input['DMI'],
                                       diet_data['Dt_NDFIn'],
                                       diet_data['Dt_StIn'],
                                       diet_data['Dt_CPIn'],
                                       diet_data['Dt_ADFIn'],
                                       diet_data['Dt_ForWet'])
                                       
    Rum_dcSt = nd.calculate_Rum_dcSt(animal_input['DMI'],
                                     diet_data['Dt_ForNDF'],
                                     diet_data['Dt_StIn'],
                                     diet_data['Dt_ForWet'])
    
    # Rumen Digestable Intakes
    Rum_DigNDFIn = nd.calculate_Rum_DigNDFIn(Rum_dcNDF, 
                                             diet_data['Dt_NDFIn'])

    Rum_DigStIn = nd.calculate_Rum_DigStIn(Rum_dcSt,
                                           diet_data['Dt_StIn'])

########################################
# Step 6: Microbial Protein Calculations
########################################
    # Infusions need to be added to the model to calculate the "An_" level nutrient intakes
    # Values are calculated here temporarily but need to be updated in the future
    An_RDPIn = diet_data['Dt_RDPIn'].copy()
    An_RDP = An_RDPIn / (animal_input['DMI'] * 100)
    An_RDPIn_g = An_RDPIn * 1000

    if equation_selection['MiN_eqn'] == 1:
        RDPIn_MiNmax = nd.calculate_RDPIn_MiNmax(animal_input['DMI'],
                                                An_RDP,
                                                An_RDPIn)
        MiN_Vm = nd.calculate_MiN_Vm(RDPIn_MiNmax, 
                                    nd.coeff_dict)
        Du_MiN_g = nd.calculate_Du_MiN_NRC2021_g(MiN_Vm,
                                                 Rum_DigNDFIn,
                                                 Rum_DigStIn,
                                                 An_RDPIn_g,
                                                 nd.coeff_dict)
    elif equation_selection['MiN_eqn'] == 2:
        Du_MiN_g = nd.calculate_Du_MiN_VTln_g(diet_data['Dt_rOMIn'],
                                              diet_data['Dt_ForNDFIn'],
                                              An_RDPIn,
                                              Rum_DigStIn,
                                              Rum_DigNDFIn,
                                              nd.coeff_dict)
    elif equation_selection['MiN_eqn'] == 3:
        Du_MiN_g = nd.calculate_Du_MiN_VTnln_g(An_RDPIn,
                                               Rum_DigNDFIn,
                                               Rum_DigStIn)
    else:
        raise ValueError(f"Invalid MiN_eqn: {equation_selection['MiN_eqn']} was entered. Must choose 1, 2 or 3.")

    Du_MiCP_g = nd.calculate_Du_MiCP_g(Du_MiN_g)
    Du_MiTP_g = nd.calculate_Du_MiTP_g(Du_MiCP_g, nd.coeff_dict)

########################################
# Step 7: Amino Acid Calculations
########################################
    AA_list = ['Arg', 'His', 'Ile', 'Leu', 'Lys', 'Met', 'Phe', 'Thr', 'Trp', 'Val']
    AA_values = pd.DataFrame(index=AA_list)
    # Dataframe for storing all individual amino acid values 

    AA_values['Du_AAMic'] = nd.calculate_Du_AAMic(Du_MiTP_g, 
                                                  AA_list, 
                                                  nd.coeff_dict)
    AA_values['Du_IdAAMic'] = nd.calculate_Du_IdAAMic(AA_values['Du_AAMic'], 
                                                      nd.coeff_dict)
    AA_values['Abs_AA_g'] = nd.calculate_Abs_AA_g(diet_data, 
                                                  AA_values['Du_IdAAMic'], 
                                                  AA_list)
    AA_values['mPrtmx_AA'] = nd.calculate_mPrtmx_AA(AA_list, 
                                                    nd.coeff_dict)
    f_mPrt_max = nd.calculate_f_mPrt_max(animal_input['An_305RHA_MlkTP'], 
                                         nd.coeff_dict)
    AA_values['mPrtmx_AA2'] = nd.calculate_mPrtmx_AA2(AA_values['mPrtmx_AA'], 
                                                      f_mPrt_max)
    AA_values['AA_mPrtmx'] = nd.calculate_AA_mPrtmx(AA_list, 
                                                    nd.coeff_dict)
    AA_values['mPrt_AA_01'] = nd.calculate_mPrt_AA_01(AA_values['AA_mPrtmx'], 
                                                      AA_list, 
                                                      nd.coeff_dict)
    AA_values['mPrt_k_AA'] = nd.calculate_mPrt_k_AA(AA_values['mPrtmx_AA2'], 
                                                    AA_values['mPrt_AA_01'], 
                                                    AA_values['AA_mPrtmx'])
   
    return animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data

### RUN MODEL ###
animal_input, diet_info, equation_selection, diet_data, AA_values, infusion_data = run_NASEM()


C:\Users\Braed\Documents\Python Projects\NASEM-Model-Python\src\nasem_dairy\NASEM_equations\dev_nutrient_intakes.py:811: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  complete_diet_info[f"{column_name}In"] = complete_diet_info.apply(lambda row: row['Fd_DMIn'] * row[f"{column_name}"], axis=1)
C:\Users\Braed\Documents\Python Projects\NASEM-Model-Python\src\nasem_dairy\NASEM_equations\dev_nutrient_intakes.py:811: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  complete_diet_info[f"{column_name}In"] = complete_diet_info.apply(lambda r